In [1]:
# Import packages
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
#import libpysal as ps
#from statsmodels.tsa.seasonal import STL

In [2]:
all_visits = pd.read_csv("raw_data/poi_visits.csv",parse_dates=['datetime'])

In [3]:
df0 = all_visits.copy()

df0['weekday'] = df0['datetime'].dt.weekday # Monday=0, Sunday=6.
df0['time'] = df0['datetime'].dt.time
df0['date'] = df0['datetime'].dt.date
df0

,Unnamed: 0.1,Unnamed: 0,uid,POI_index,datetime,type,lat,lng,home_pings,home_tile_ID,work_pings,work_tile_ID,tot_pings,month,distance,weekday,day,stratum,time,date
0,64893,64893,077480D3-7C5E-46AE-8CAF-E949E78C5A35,4201,2020-01-01 00:00:00-05:00,bars,4.673600,-74.080800,344,114,100,112,10996,1,5899.702473,2,1,4.675979,00:00:00,2020-01-01
1,64933,64933,077480D3-7C5E-46AE-8CAF-E949E78C5A35,2185,2020-01-01 00:00:00-05:00,dentists,4.673611,-74.080810,344,114,100,112,10996,1,5898.887773,2,1,4.675979,00:00:00,2020-01-01
2,65522,65522,077480d3-7c5e-46ae-8caf-e949e78c5a35,1754,2020-01-01 00:00:00-05:00,conveniences,4.676800,-74.081300,174,114,79,112,6160,1,5643.532653,2,1,4.675979,00:00:00,2020-01-01
3,65550,65550,077480d3-7c5e-46ae-8caf-e949e78c5a35,2537,2020-01-01 00:00:00-05:00,hgs,4.673000,-74.080000,174,114,79,112,6160,1,5992.393360,2,1,4.675979,00:00:00,2020-01-01
4,521670,521670,3f5e8df1-da24-4259-a16c-d34cc31f9bb2,7772,2020-01-01 00:00:00-05:00,restaurants,4.853300,-74.056700,70,120,538,229,3081,1,12217.889000,2,1,3.631530,00:00:00,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,180562,180562,14fb39bc-e5a8-4638-97a9-7602d8a64229,471,2021-06-30 18:36:00-05:00,cloths,4.603200,-74.190700,0,526,9,526,11,18,13340.905060,2,546,2.951414,18:36:00,2021-06-30
1048571,262726,262726,1f064ba0-201f-4c25-8105-aa2738c8f11b,1505,2021-06-30 18:36:00-05:00,restaurants,4.587000,-74.206400,0,158,11,158,73,18,1283.696520,2,546,2.385780,18:36:00,2021-06-30
1048572,112412,112412,0cd97ae5-df49-455f-9bda-9c7b5f36659e,4238,2021-06-30 18:48:00-05:00,cafes,4.598300,-74.071700,3,598,323,598,1218,18,6030.608988,2,546,2.997531,18:48:00,2021-06-30
1048573,180563,180563,14fb39bc-e5a8-4638-97a9-7602d8a64229,471,2021-06-30 18:48:00-05:00,cloths,4.603200,-74.190700,0,526,9,526,11,18,13340.905060,2,546,2.951414,18:48:00,2021-06-30


In [4]:
# select visit during opening time
df0 = df0.loc[(df0['time']> pd.to_datetime('2020-01-01 08:00:00').time())&
              (df0['time']< pd.to_datetime('2020-01-01 20:00:00').time())]
        
df0.reset_index()

,index,Unnamed: 0.1,Unnamed: 0,uid,POI_index,datetime,type,lat,lng,home_pings,...,work_pings,work_tile_ID,tot_pings,month,distance,weekday,day,stratum,time,date
0,225,490311,490311,3ae88035-01d3-4c61-aa94-b8dd733b9637,9043,2020-01-01 08:12:00-05:00,restaurants,4.639700,-74.154000,0,...,8,488,165,1,3040.614396,2,1,2.322873,08:12:00,2020-01-01
1,226,762631,762631,5bd8e4fe-f975-4084-a2e4-7eb3f04d9fd2,7861,2020-01-01 08:12:00-05:00,hgs,4.578600,-74.098300,15,...,161,169,2319,1,10702.027450,2,1,2.019861,08:12:00,2020-01-01
2,227,1036139,1036139,80cde304-9dd6-42c7-8b7f-135143066673,7424,2020-01-01 08:12:00-05:00,bars,4.670700,-74.070500,17,...,108,234,497,1,13288.476780,2,1,2.879417,08:12:00,2020-01-01
3,228,97444,97444,0b296a43-3e39-403c-96f0-519783163f3c,4238,2020-01-01 08:24:00-05:00,conveniences,4.732200,-74.030300,0,...,5,776,35,1,1112.338212,2,1,2.989511,08:24:00,2020-01-01
4,229,365640,365640,2a777f49-cd7b-4a33-8cb5-d56caa1eba73,4074,2020-01-01 08:24:00-05:00,hgs,4.728000,-74.088600,150,...,1991,89,7334,1,4511.490313,2,1,2.013404,08:24:00,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850270,1048570,180562,180562,14fb39bc-e5a8-4638-97a9-7602d8a64229,471,2021-06-30 18:36:00-05:00,cloths,4.603200,-74.190700,0,...,9,526,11,18,13340.905060,2,546,2.951414,18:36:00,2021-06-30
850271,1048571,262726,262726,1f064ba0-201f-4c25-8105-aa2738c8f11b,1505,2021-06-30 18:36:00-05:00,restaurants,4.587000,-74.206400,0,...,11,158,73,18,1283.696520,2,546,2.385780,18:36:00,2021-06-30
850272,1048572,112412,112412,0cd97ae5-df49-455f-9bda-9c7b5f36659e,4238,2021-06-30 18:48:00-05:00,cafes,4.598300,-74.071700,3,...,323,598,1218,18,6030.608988,2,546,2.997531,18:48:00,2021-06-30
850273,1048573,180563,180563,14fb39bc-e5a8-4638-97a9-7602d8a64229,471,2021-06-30 18:48:00-05:00,cloths,4.603200,-74.190700,0,...,9,526,11,18,13340.905060,2,546,2.951414,18:48:00,2021-06-30


In [5]:
shp = gpd.read_file("raw_data/shp/zat_stratum.shp")

In [6]:
df1 =  df0.merge(shp, left_on='home_tile_ID', right_on='ZAT', how = 'left')
df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 850275 entries, 0 to 850274
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype                                 
---  ------        --------------   -----                                 
 0   Unnamed: 0.1  850275 non-null  int64                                 
 1   Unnamed: 0    850275 non-null  int64                                 
 2   uid           850275 non-null  object                                
 3   POI_index     850275 non-null  int64                                 
 4   datetime      850275 non-null  datetime64[ns, pytz.FixedOffset(-300)]
 5   type          850275 non-null  object                                
 6   lat           850275 non-null  float64                               
 7   lng           850275 non-null  float64                               
 8   home_pings    850275 non-null  int64                                 
 9   home_tile_ID  850275 non-null  int64                       

In [7]:
# 256(adino), 242(atlantis plaza)
#df1 = df1.loc[(df1['work_tile_ID'] == 256) | (df1['work_tile_ID'] == 242)]


In [8]:
by_tp = pd.DataFrame({"poi":df1["POI_index"],"day":df1["datetime"].dt.dayofyear+366*(df1["datetime"].dt.year==2021),
                      "date":df1["date"],"datetime":df1["datetime"], "weekday":df1["weekday"],"time":df1["time"],"dist":df1["distance"],
                      "type":df1["type"],"uid":df1["uid"], "lat":df1["lat"], "lng":df1["lng"],
                      "from_ID":df1["home_tile_ID"], "from_stra": df1["stratum_y"], "to_ID":df1["work_tile_ID"], "to_stra":df1["stratum_x"]})

by_tp = by_tp.drop_duplicates(subset=["day","poi","uid"])
by_tp = by_tp.loc[by_tp['day']<=366]



In [15]:
by_tp.to_csv('toR.csv')

In [9]:
strat = pd.read_csv("raw_data/strat_stats.csv")
strat


,day,count,av,median,low_ct,med_ct,hi_ct
0,1,557118,2.824008,2.839070,230467,297584,29067
1,2,442660,2.928896,2.899845,173067,234201,35392
2,3,356399,3.023536,2.951414,126606,195036,34757
3,4,424599,2.940255,2.915527,163046,226168,35385
4,5,333147,3.028093,2.938682,115740,182310,35097
...,...,...,...,...,...,...,...
542,543,354710,2.749812,2.781147,146207,193295,15208
543,544,285929,2.694051,2.678130,130061,142935,12933
544,545,239117,2.787076,2.875949,94958,132651,11508
545,546,144807,2.724574,2.781147,61698,76955,6154


In [10]:

#by_tp

In [11]:
by_tp['datetime'] = pd.to_datetime(by_tp['datetime'])

df2 = by_tp.groupby(['from_ID', 'day']).count()["uid"].reset_index()
df2 = df2.merge(strat, left_on='day', right_on='day')
df2['ratio'] = df2.apply(lambda x:x["uid"]/x["count"],axis=1)
df2

,from_ID,day,uid,count,av,median,low_ct,med_ct,hi_ct,ratio
0,0,4,11,424599,2.940255,2.915527,163046,226168,35385,0.000026
1,2,4,6,424599,2.940255,2.915527,163046,226168,35385,0.000014
2,3,4,5,424599,2.940255,2.915527,163046,226168,35385,0.000012
3,6,4,3,424599,2.940255,2.915527,163046,226168,35385,0.000007
4,7,4,3,424599,2.940255,2.915527,163046,226168,35385,0.000007
...,...,...,...,...,...,...,...,...,...,...
116770,947,257,2,203636,2.781646,2.781147,85738,107225,10673,0.000010
116771,973,257,1,203636,2.781646,2.781147,85738,107225,10673,0.000005
116772,999,257,1,203636,2.781646,2.781147,85738,107225,10673,0.000005
116773,1076,257,1,203636,2.781646,2.781147,85738,107225,10673,0.000005


In [12]:
df2.to_csv('from_ZAT_day.csv')

In [13]:
print(df2.loc[df2['day'] == 164])
print(df2.loc[df2['day'] == 171])
print(df2.loc[df2['day'] == 178])

       from_ID  day  uid    count       av    median  low_ct  med_ct  hi_ct  \
85764        2  164    1  1248243  2.76712  2.781147  519035  666012  63196   
85765        7  164    1  1248243  2.76712  2.781147  519035  666012  63196   
85766       12  164   10  1248243  2.76712  2.781147  519035  666012  63196   
85767       16  164    1  1248243  2.76712  2.781147  519035  666012  63196   
85768       17  164    1  1248243  2.76712  2.781147  519035  666012  63196   
...        ...  ...  ...      ...      ...       ...     ...     ...    ...   
86236     1079  164    2  1248243  2.76712  2.781147  519035  666012  63196   
86237     1081  164    1  1248243  2.76712  2.781147  519035  666012  63196   
86238     1088  164    1  1248243  2.76712  2.781147  519035  666012  63196   
86239     1094  164    1  1248243  2.76712  2.781147  519035  666012  63196   
86240     1095  164   23  1248243  2.76712  2.781147  519035  666012  63196   

              ratio  
85764  8.011261e-07  
85765  